In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'c:\\datascience\\mypractice\\Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    output_dir: Path
    evaluation_strategy: str
    learning_rate: float
    num_train_epochs: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    warmup_steps: int
    weight_decay: float
    save_total_limit: int
    gradient_accumulation_steps: int
    logging_dir: Path
    save_strategy: str
    load_best_model_at_end: bool
    metric_for_best_model: str
    greater_is_better: bool
    fp16: bool    

In [7]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            output_dir = params.output_dir,
            evaluation_strategy = params.evaluation_strategy,
            learning_rate = params.learning_rate,
            num_train_epochs = params.num_train_epochs,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            warmup_steps = params.warmup_steps,
            weight_decay = params.weight_decay,
            save_total_limit = params.save_total_limit,
            gradient_accumulation_steps = params.gradient_accumulation_steps,
            logging_dir = params.logging_dir,
            save_strategy = params.save_strategy,
            load_best_model_at_end = params.load_best_model_at_end,
            metric_for_best_model = params.metric_for_best_model,
            greater_is_better = params.greater_is_better,
            fp16 = params.fp16
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
torch.cuda.is_available()

c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[ 2024-05-04 20:21:18,779 ] 58 datasets - INFO - PyTorch version 2.3.0+cu118 available.


True

In [7]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [10]:
! pip install -U accelerate
! pip install -U transformers

  Using cached accelerate-0.30.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-0.30.0-py3-none-any.whl (302 kB)
  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.39.1
    Uninstalling transformers-4.39.1:
      Successfully uninstalled transformers-4.39.1


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textsummarizer 0.0.0 requires torch==2.2.1, but you have torch 2.3.0+cu118 which is incompatible.
textsummarizer 0.0.0 requires transformers==4.39.1, but you have transformers 4.40.1 which is incompatible.
textsummarizer 0.0.0 requires transformers[sentencepiece]==4.39.1, but you have transformers 4.40.1 which is incompatible.


In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=os.path.join(self.config.root_dir, self.config.output_dir), 
            evaluation_strategy=self.config.evaluation_strategy,
            learning_rate=float(self.config.learning_rate),
            num_train_epochs=self.config.num_train_epochs,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_eval_batch_size,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            save_total_limit=self.config.save_total_limit,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
            logging_dir=self.config.logging_dir,
            save_strategy=self.config.save_strategy,
            load_best_model_at_end=self.config.load_best_model_at_end,
            metric_for_best_model=self.config.metric_for_best_model,
            greater_is_better=self.config.greater_is_better,
            fp16=self.config.fp16
        )


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"t5-small-samsum"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

